<a href="https://colab.research.google.com/github/juvascoli/redes-neurais-keras/blob/main/cp_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EXERCÍCIO 1 – CLASSIFICAÇÃO MULTICLASSE
Dataset: Wine Dataset (UCI)
1. Treinar uma rede neural em Keras para classificar vinhos em 3 classes.
- Configuração mínima: 2 camadas ocultas com 32 neurônios cada, função de ativação ReLU.
- Camada de saída com 3 neurônios, função de ativação Softmax.
- Função de perda: categorical_crossentropy.
- Otimizador: Adam.
2. Comparar os resultados com um modelo do scikit-learn (RandomForestClassifier ou
LogisticRegression).
3. Registrar métricas de acurácia e discutir qual modelo teve melhor desempenho.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Carregar o dataset
cols = ['Class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium',
        'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols', 'Proanthocyanins',
        'Color_intensity', 'Hue', 'OD280_OD315', 'Proline']
df = pd.read_csv('wine.data', header=None, names=cols)

X = df.drop('Class', axis=1).values
y = df['Class'].values - 1  # classes: 0, 1, 2

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42)

# Normalizar features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Definir e treinar o modelo
model = Sequential([
    Dense(32, activation='relu', input_shape=(13,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train_cat, epochs=50, batch_size=8,
                    validation_split=0.1, verbose=1)

score = model.evaluate(X_test_scaled, y_test_cat, verbose=0)
print(f'Acurácia Rede Neural (Keras): {score[1]:.4f}')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_pred)
print(f'Acurácia RandomForest: {rf_acc:.4f}')

# LogisticRegression
lr = LogisticRegression(max_iter=200, random_state=42)
lr.fit(X_train_scaled, y_train)
lr_pred = lr.predict(X_test_scaled)
lr_acc = accuracy_score(y_test, lr_pred)
print(f'Acurácia LogisticRegression: {lr_acc:.4f}')

In [ ]:
results = pd.DataFrame({
    'Modelo': ['Rede Neural (Keras)', 'RandomForest', 'LogisticRegression'],
    'Acurácia': [score[1], rf_acc, lr_acc]
})
print(results)

EXERCÍCIO 2 – REGRESSÃO
Dataset: California Housing Dataset (Scikit-learn)
1. Treinar uma rede neural em Keras para prever o valor médio das casas.
- Configuração mínima: 3 camadas ocultas com 64, 32 e 16 neurônios, função de ativação ReLU.
- Camada de saída com 1 neurônio, função de ativação Linear.
- Função de perda: mse.
- Otimizador: Adam.
2. Comparar os resultados com um modelo do scikit-learn (LinearRegression ou
RandomForestRegressor).
3. Registrar métricas de erro (RMSE ou MAE) e discutir qual modelo teve melhor desempenho.


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Carregar o dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Normalizar features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Definir e treinar o modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(X_train_scaled, y_train, epochs=25, batch_size=32,
                    validation_split=0.1, verbose=1)

# Avaliação
loss, mae = model.evaluate(X_test_scaled, y_test, verbose=0)
y_pred_nn = model.predict(X_test_scaled).ravel()
rmse_nn = np.sqrt(np.mean((y_pred_nn - y_test) ** 2))
print(f'MAE (Rede Neural): {mae:.3f}')
print(f'RMSE (Rede Neural): {rmse_nn:.3f}')


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print(f'MAE (LinearRegression): {mae_lr:.3f}')
print(f'RMSE (LinearRegression): {rmse_lr:.3f}')

# RandomForest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f'MAE (RandomForestRegressor): {mae_rf:.3f}')
print(f'RMSE (RandomForestRegressor): {rmse_rf:.3f}')


In [ ]:
import pandas as pd

result = pd.DataFrame(
    {
        "Modelo": [
            "Rede Neural (Keras)",
            "LinearRegression",
            "RandomForestRegressor"
        ],
        "MAE": [mae, mae_lr, mae_rf],
        "RMSE": [rmse_nn, rmse_lr, rmse_rf]
    }
)
print(result)
